In [269]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import seaborn as sns

In [270]:
pip install lxml

Note: you may need to restart the kernel to use updated packages.


In [271]:
output_dir = "/kaggle/working/data"
os.makedirs(output_dir, exist_ok=True)

In [272]:
import os
print("Datasets in /kaggle/input/:", os.listdir("/kaggle/input/"))

Datasets in /kaggle/input/: ['sysmon-csv-ds', 'sysmon']


In [ ]:
# path=r'/kaggle/input/sysmon/sysmon ds'
import xml.etree.ElementTree as ET
import csv
def loader(path,label):
    for i in os.listdir(path):
        file_path=path+'/'+i
        tree=ET.parse(file_path)
        root=tree.getroot()
        events=[]
        for event in root.findall("Event"):
            system=event.find("System")
            event_data=event.find("EventData")
            event_id = system.find("EventID").text if system.find("EventID") is not None else None
            utc_time = event_data.find("Data[@Name='UtcTime']").text if event_data.find("Data[@Name='UtcTime']") is not None else None
            process_guid = event_data.find("Data[@Name='ProcessGuid']").text if event_data.find("Data[@Name='ProcessGuid']") is not None else None
            process_id = event_data.find("Data[@Name='ProcessId']").text if event_data.find("Data[@Name='ProcessId']") is not None else None
            image = event_data.find("Data[@Name='Image']").text if event_data.find("Data[@Name='Image']") is not None else None
            command_line = event_data.find("Data[@Name='CommandLine']").text if event_data.find("Data[@Name='CommandLine']") is not None else None
            parent_image = event_data.find("Data[@Name='ParentImage']").text if event_data.find("Data[@Name='ParentImage']") is not None else None
            user = event_data.find("Data[@Name='User']").text if event_data.find("Data[@Name='User']") is not None else None
            integrity_level = event_data.find("Data[@Name='IntegrityLevel']").text if event_data.find("Data[@Name='IntegrityLevel']") is not None else None
            hashes = event_data.find("Data[@Name='Hashes']").text if event_data.find("Data[@Name='Hashes']") is not None else None
            # Append extracted data to events list
            events.append([event_id, utc_time, process_guid, process_id, image, command_line, parent_image, user, integrity_level, hashes,label])
        with open(f"/kaggle/working/data/{i}.csv","w",newline="") as f:
            writer=csv.writer(f)
            writer.writerow(["event_id","utc_time","process_guid","process_id","image","command_line","parent_image","user","integrity_level","hashes","label"])
            writer.writerows(events)
        print("file")
loader(r"/kaggle/input/sysmon/sysmon ds/benign",1)
loader(r"/kaggle/input/sysmon/sysmon ds/mal",0)

'''
import os
import xml.etree.ElementTree as ET
import csv
from datetime import datetime

# Function to parse XML with namespaces
def parse_sysmon_xml(file_path, label):
    events = []
    
    try:
        tree = ET.parse(file_path)
        root = tree.getroot()

        # Define the namespace (Sysmon logs use a default XML namespace)
        namespace = {'ns': 'http://schemas.microsoft.com/win/2004/08/events/event'}

        for event in root.findall("ns:Event", namespace):
            system = event.find("ns:System", namespace)
            event_data = event.find("ns:EventData", namespace)

            if system is None or event_data is None:
                continue  # Skip if the structure is missing

            # Extract relevant fields
            event_id = system.find("ns:EventID", namespace).text if system.find("ns:EventID", namespace) is not None else None
            #utc_time = event_data.find("ns:Data[@Name='UtcTime']", namespace).text if event_data.find("ns:Data[@Name='UtcTime']", namespace) is not None else None
            utc_time = event_data.find("ns:Data[@Name='UtcTime']", namespace).text if event_data.find("ns:Data[@Name='UtcTime']", namespace) is not None else None
            if utc_time:
                try:
                # Convert to a datetime object to ensure the format is consistent (and adjust format if necessary)
                    utc_time = datetime.strptime(utc_time, "%Y-%m-%dT%H:%M:%S.%fZ").strftime("%Y-%m-%d %H:%M:%S")
                except ValueError:
                    pass  # Handle any parsing issues (optional)
            process_guid = event_data.find("ns:Data[@Name='ProcessGuid']", namespace).text if event_data.find("ns:Data[@Name='ProcessGuid']", namespace) is not None else None
            process_id = event_data.find("ns:Data[@Name='ProcessId']", namespace).text if event_data.find("ns:Data[@Name='ProcessId']", namespace) is not None else None
            image = event_data.find("ns:Data[@Name='Image']", namespace).text if event_data.find("ns:Data[@Name='Image']", namespace) is not None else None
            command_line = event_data.find("ns:Data[@Name='CommandLine']", namespace).text if event_data.find("ns:Data[@Name='CommandLine']", namespace) is not None else None
            parent_image = event_data.find("ns:Data[@Name='ParentImage']", namespace).text if event_data.find("ns:Data[@Name='ParentImage']", namespace) is not None else None
            user = event_data.find("ns:Data[@Name='User']", namespace).text if event_data.find("ns:Data[@Name='User']", namespace) is not None else None
            integrity_level = event_data.find("ns:Data[@Name='IntegrityLevel']", namespace).text if event_data.find("ns:Data[@Name='IntegrityLevel']", namespace) is not None else None
            hashes = event_data.find("ns:Data[@Name='Hashes']", namespace).text if event_data.find("ns:Data[@Name='Hashes']", namespace) is not None else None

            # Append extracted data to events list
            events.append([event_id, utc_time, process_guid, process_id, image, command_line, parent_image, user, integrity_level, hashes, label])
    
    except Exception as e:
        print(f"Error processing {file_path}: {e}")

    return events


# Function to process multiple XML files and save as CSV
def process_directory(input_folder, label, output_csv):
    all_events = []

    for file_name in os.listdir(input_folder):
        if file_name.endswith(".xml"):  # Process only XML files
            file_path = os.path.join(input_folder, file_name)
            print(f"Processing: {file_path}")
            all_events.extend(parse_sysmon_xml(file_path, label))

    # Save extracted data to CSV
    with open(output_csv, "w", newline="") as f:
        writer = csv.writer(f)
        writer.writerow(["event_id", "utc_time", "process_guid", "process_id", "image", "command_line", "parent_image", "user", "integrity_level", "hashes", "label"])
        writer.writerows(all_events)
    
    print(f"CSV file saved: {output_csv}")


# Create output folder if it doesn't exist
os.makedirs("/kaggle/working/data", exist_ok=True)

# Process both benign and malicious logs
process_directory("/kaggle/input/sysmon/sysmon ds/mal", 1, "/kaggle/working/data/malicious_logs.csv")
process_directory("/kaggle/input/sysmon/sysmon ds/benign", 0, "/kaggle/working/data/benign_logs.csv")
'''

file
file
file
file
file
file
file
file
file
file
file
file
file
file
file
file
file
file
file
file
file
file
file
file
file
file


'\nimport os\nimport xml.etree.ElementTree as ET\nimport csv\nfrom datetime import datetime\n\n# Function to parse XML with namespaces\ndef parse_sysmon_xml(file_path, label):\n    events = []\n    \n    try:\n        tree = ET.parse(file_path)\n        root = tree.getroot()\n\n        # Define the namespace (Sysmon logs use a default XML namespace)\n        namespace = {\'ns\': \'http://schemas.microsoft.com/win/2004/08/events/event\'}\n\n        for event in root.findall("ns:Event", namespace):\n            system = event.find("ns:System", namespace)\n            event_data = event.find("ns:EventData", namespace)\n\n            if system is None or event_data is None:\n                continue  # Skip if the structure is missing\n\n            # Extract relevant fields\n            event_id = system.find("ns:EventID", namespace).text if system.find("ns:EventID", namespace) is not None else None\n            #utc_time = event_data.find("ns:Data[@Name=\'UtcTime\']", namespace).text if 

In [274]:
beni=pd.read_csv(r'/kaggle/input/sysmon-csv-ds/benign_logs.csv')
beni.shape

(318527, 11)

In [275]:
mal=pd.read_csv(r'/kaggle/input/sysmon-csv-ds/malicious_logs.csv')
mal.shape

(10789, 11)

In [276]:
beni.columns

Index(['event_id', 'utc_time', 'process_guid', 'process_id', 'image',
       'command_line', 'parent_image', 'user', 'integrity_level', 'hashes',
       'label'],
      dtype='object')

## starting with an NN model, for basic classification. time wise data for LSTM can be seen next

# Pre Processing considerations

## since 3-4 columns have NaN values of almost 50% entries, we must consider considerations to prevent bias and overfitting

### for parent image : embeddings, fill NaN as "Unknown_image"
### for command_line : keep the category unknown/NoCommand, convert to embeddings
### len(command_line) : if size is high, too many commands are being sent. take token count for this
### hash_exists : 1(yes) or 0(no)
### has_parent_image : 1(yes) or 0(no)
### integrity : keep everything medium if they are missing (usual value)

In [277]:
mal["process_guid"].value_counts()

process_guid
{558F98EB-42A7-62D7-C500-000000004600}    2
{558F98EB-C046-62D6-F000-000000003B00}    2
{558F98EB-C0BF-62D6-0A01-000000003B00}    2
{558F98EB-C0BF-62D6-0B01-000000003B00}    2
{558F98EB-C0BF-62D6-0C01-000000003B00}    2
                                         ..
{558F98EB-B45B-62D6-4700-000000003A00}    1
{558F98EB-B4DD-62D6-5E00-000000003A00}    1
{558F98EB-B4DD-62D6-5F00-000000003A00}    1
{558F98EB-B44B-62D6-1600-000000003A00}    1
{558F98EB-D53F-62D6-9000-000000003D00}    1
Name: count, Length: 5624, dtype: int64

In [278]:
df = pd.concat([beni, mal], ignore_index=True)

In [279]:
df

,event_id,utc_time,process_guid,process_id,image,command_line,parent_image,user,integrity_level,hashes,label
0,5,2022-07-19 16:01:43.056,{558F98EB-D567-62D6-B519-000000005800},1688.0,C:\Program Files\Confer\RepWmiUtils.exe,NaN,NaN,NT AUTHORITY\SYSTEM,NaN,NaN,0
1,1,2022-07-19 16:01:43.002,{558F98EB-D567-62D6-B519-000000005800},1688.0,C:\Program Files\Confer\RepWmiUtils.exe,RepWmiUtils.exe muuid,-,NT AUTHORITY\SYSTEM,System,SHA256=11F4F9472E174D22264B8C2D3D0834E699B882D...,0
2,5,2022-07-19 16:01:42.998,{558F98EB-D566-62D6-B419-000000005800},512.0,C:\Program Files\Confer\RepWmiUtils.exe,NaN,NaN,NT AUTHORITY\SYSTEM,NaN,NaN,0
3,1,2022-07-19 16:01:42.937,{558F98EB-D566-62D6-B419-000000005800},512.0,C:\Program Files\Confer\RepWmiUtils.exe,RepWmiUtils.exe muuid,-,NT AUTHORITY\SYSTEM,System,SHA256=11F4F9472E174D22264B8C2D3D0834E699B882D...,0
4,5,2022-07-19 16:01:42.847,{558F98EB-D566-62D6-B319-000000005800},4920.0,C:\Program Files\Confer\RepWmiUtils.exe,NaN,NaN,NT AUTHORITY\SYSTEM,NaN,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...
329311,1,2022-07-19 16:01:46.219,{558F98EB-D56A-62D6-9900-000000003D00},1892.0,C:\Windows\System32\conhost.exe,"\??\C:\Windows\system32\conhost.exe ""-46834291...",-,WIN-32-H1\aalsahee,Medium,SHA256=86917F7235AD75A6B9F8DA00D6B1A4040C8D857...,1
329312,1,2022-07-19 16:01:46.204,{558F98EB-D56A-62D6-9800-000000003D00},2872.0,C:\Windows\System32\cmd.exe,"C:\Windows\system32\cmd.exe /c """"C:\Users\aals...",-,WIN-32-H1\aalsahee,Medium,SHA256=17F746D82695FA9B35493B41859D39D786D32B2...,1
329313,5,2022-07-19 16:01:46.017,{558F98EB-D4ED-62D6-6F00-000000003D00},1504.0,C:\Windows\System32\SearchFilterHost.exe,NaN,NaN,NT AUTHORITY\SYSTEM,NaN,NaN,1
329314,5,2022-07-19 16:01:46.001,{558F98EB-D4ED-62D6-6E00-000000003D00},1404.0,C:\Windows\System32\SearchProtocolHost.exe,NaN,NaN,NT AUTHORITY\SYSTEM,NaN,NaN,1


In [280]:
df.loc[:, 'parent_image'] = df['parent_image'].fillna("Unknown_image")
df.loc[:, 'command_line'] = df['command_line'].fillna("NoCommand")
df.loc[:, 'integrity_level'] = df['integrity_level'].fillna("Medium")

In [281]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
df['parent_image_encoded'] = encoder.fit_transform(df['parent_image'])
df['command_line_encoded'] = encoder.fit_transform(df['command_line'])

In [282]:
df['command_length'] = df['command_line'].apply(lambda x: len(x.split()))
df['hash_exists'] = df['hashes'].notna().astype(int) 
df['has_parent_image'] = df['parent_image'].apply(lambda x: 0 if x == "Unknown_image" else 1) 

In [283]:
df.drop(columns=['parent_image', 'command_line', 'hashes'], inplace=True)

In [284]:
df

,event_id,utc_time,process_guid,process_id,image,user,integrity_level,label,parent_image_encoded,command_line_encoded,command_length,hash_exists,has_parent_image
0,5,2022-07-19 16:01:43.056,{558F98EB-D567-62D6-B519-000000005800},1688.0,C:\Program Files\Confer\RepWmiUtils.exe,NT AUTHORITY\SYSTEM,Medium,0,55,1236,1,0,0
1,1,2022-07-19 16:01:43.002,{558F98EB-D567-62D6-B519-000000005800},1688.0,C:\Program Files\Confer\RepWmiUtils.exe,NT AUTHORITY\SYSTEM,System,0,0,1237,2,1,1
2,5,2022-07-19 16:01:42.998,{558F98EB-D566-62D6-B419-000000005800},512.0,C:\Program Files\Confer\RepWmiUtils.exe,NT AUTHORITY\SYSTEM,Medium,0,55,1236,1,0,0
3,1,2022-07-19 16:01:42.937,{558F98EB-D566-62D6-B419-000000005800},512.0,C:\Program Files\Confer\RepWmiUtils.exe,NT AUTHORITY\SYSTEM,System,0,0,1237,2,1,1
4,5,2022-07-19 16:01:42.847,{558F98EB-D566-62D6-B319-000000005800},4920.0,C:\Program Files\Confer\RepWmiUtils.exe,NT AUTHORITY\SYSTEM,Medium,0,55,1236,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
329311,1,2022-07-19 16:01:46.219,{558F98EB-D56A-62D6-9900-000000003D00},1892.0,C:\Windows\System32\conhost.exe,WIN-32-H1\aalsahee,Medium,1,0,1457,2,1,1
329312,1,2022-07-19 16:01:46.204,{558F98EB-D56A-62D6-9800-000000003D00},2872.0,C:\Windows\System32\cmd.exe,WIN-32-H1\aalsahee,Medium,1,0,1191,4,1,1
329313,5,2022-07-19 16:01:46.017,{558F98EB-D4ED-62D6-6F00-000000003D00},1504.0,C:\Windows\System32\SearchFilterHost.exe,NT AUTHORITY\SYSTEM,Medium,1,55,1236,1,0,0
329314,5,2022-07-19 16:01:46.001,{558F98EB-D4ED-62D6-6E00-000000003D00},1404.0,C:\Windows\System32\SearchProtocolHost.exe,NT AUTHORITY\SYSTEM,Medium,1,55,1236,1,0,0


In [285]:
df.drop(columns=["process_guid"], inplace=True)

### process_guid...too unique for categorizing, so drop it

In [286]:
df.isnull().sum()

event_id                 0
utc_time                 0
process_id              11
image                   11
user                    11
integrity_level          0
label                    0
parent_image_encoded     0
command_line_encoded     0
command_length           0
hash_exists              0
has_parent_image         0
dtype: int64

In [287]:
mal["image"].value_counts()

image
C:\Program Files\Confer\RepWmiUtils.exe                                 7777
C:\Windows\System32\dllhost.exe                                          487
C:\Windows\System32\conhost.exe                                          314
C:\Windows\System32\SearchProtocolHost.exe                               213
C:\Windows\System32\SearchFilterHost.exe                                 203
                                                                        ... 
C:\Windows\System32\winlogon.exe                                           4
C:\Windows\System32\spoolsv.exe                                            4
C:\Program Files\VMware\VMware Tools\VMware VGAuth\VGAuthService.exe       4
C:\Windows\System32\rundll32.exe                                           2
C:\Windows\System32\control.exe                                            2
Name: count, Length: 81, dtype: int64

## use the same Word2Vec model for image and parent_image. and we are only considered with 2 things in images

### * image depth
### * image filename

In [288]:
import os
def extract_image_features(image_path):
    """Extract image depth and filename"""
    if pd.isna(image_path):
        return 0, "Unknown_image" 
    path_parts = image_path.split("\\")
    depth = len(path_parts)
    filename = path_parts[-1] 
    return depth, filename
df["image_depth"], df["image_filename"] = zip(*df["image"].apply(extract_image_features))
df["parent_image_depth"], df["parent_image_filename"] = zip(*df["parent_image_encoded"].astype(str).apply(extract_image_features))

In [289]:
from gensim.models import Word2Vec
sentences = df["image_filename"].dropna().tolist() + df["parent_image_filename"].dropna().tolist()
sentences = [[word] for word in sentences]
w2v_model = Word2Vec(sentences, vector_size=100, window=5, min_count=1, workers=4)
def get_w2v_embedding(word):
    if word in w2v_model.wv:
        return w2v_model.wv[word]
    else:
        return [0] * 100 
df["image_embedding"] = df["image_filename"].apply(lambda x: get_w2v_embedding(x))
df["parent_image_embedding"] = df["parent_image_filename"].apply(lambda x: get_w2v_embedding(x))

In [290]:
df.drop(columns=["image", "image_filename", "parent_image_filename"], inplace=True)

In [291]:
df

,event_id,utc_time,process_id,user,integrity_level,label,parent_image_encoded,command_line_encoded,command_length,hash_exists,has_parent_image,image_depth,parent_image_depth,image_embedding,parent_image_embedding
0,5,2022-07-19 16:01:43.056,1688.0,NT AUTHORITY\SYSTEM,Medium,0,55,1236,1,0,0,4,1,"[-0.00053622725, 0.00023643136, 0.0051033497, ...","[-0.0086196875, 0.003665738, 0.0051898835, 0.0..."
1,1,2022-07-19 16:01:43.002,1688.0,NT AUTHORITY\SYSTEM,System,0,0,1237,2,1,1,4,1,"[-0.00053622725, 0.00023643136, 0.0051033497, ...","[9.456396e-05, 0.0030773198, -0.006812645, -0...."
2,5,2022-07-19 16:01:42.998,512.0,NT AUTHORITY\SYSTEM,Medium,0,55,1236,1,0,0,4,1,"[-0.00053622725, 0.00023643136, 0.0051033497, ...","[-0.0086196875, 0.003665738, 0.0051898835, 0.0..."
3,1,2022-07-19 16:01:42.937,512.0,NT AUTHORITY\SYSTEM,System,0,0,1237,2,1,1,4,1,"[-0.00053622725, 0.00023643136, 0.0051033497, ...","[9.456396e-05, 0.0030773198, -0.006812645, -0...."
4,5,2022-07-19 16:01:42.847,4920.0,NT AUTHORITY\SYSTEM,Medium,0,55,1236,1,0,0,4,1,"[-0.00053622725, 0.00023643136, 0.0051033497, ...","[-0.0086196875, 0.003665738, 0.0051898835, 0.0..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
329311,1,2022-07-19 16:01:46.219,1892.0,WIN-32-H1\aalsahee,Medium,1,0,1457,2,1,1,4,1,"[-0.007139015, 0.0012410306, -0.0071767163, -0...","[9.456396e-05, 0.0030773198, -0.006812645, -0...."
329312,1,2022-07-19 16:01:46.204,2872.0,WIN-32-H1\aalsahee,Medium,1,0,1191,4,1,1,4,1,"[0.00973555, -0.009780383, -0.00649949, 0.0027...","[9.456396e-05, 0.0030773198, -0.006812645, -0...."
329313,5,2022-07-19 16:01:46.017,1504.0,NT AUTHORITY\SYSTEM,Medium,1,55,1236,1,0,0,4,1,"[-0.0051562428, -0.0066683376, -0.007776835, 0...","[-0.0086196875, 0.003665738, 0.0051898835, 0.0..."
329314,5,2022-07-19 16:01:46.001,1404.0,NT AUTHORITY\SYSTEM,Medium,1,55,1236,1,0,0,4,1,"[0.007088797, -0.00156793, 0.007947499, -0.009...","[-0.0086196875, 0.003665738, 0.0051898835, 0.0..."


In [292]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

In [293]:
# df["utc_time"] = pd.to_datetime(df["utc_time"])
# df["hour"] = df["utc_time"].dt.hour
# df["day"] = df["utc_time"].dt.day
# df["month"] = df["utc_time"].dt.month
# df = df.drop(columns=["utc_time"], inplace=True) 

In [294]:
df

,event_id,utc_time,process_id,user,integrity_level,label,parent_image_encoded,command_line_encoded,command_length,hash_exists,has_parent_image,image_depth,parent_image_depth,image_embedding,parent_image_embedding
0,5,2022-07-19 16:01:43.056,1688.0,NT AUTHORITY\SYSTEM,Medium,0,55,1236,1,0,0,4,1,"[-0.00053622725, 0.00023643136, 0.0051033497, ...","[-0.0086196875, 0.003665738, 0.0051898835, 0.0..."
1,1,2022-07-19 16:01:43.002,1688.0,NT AUTHORITY\SYSTEM,System,0,0,1237,2,1,1,4,1,"[-0.00053622725, 0.00023643136, 0.0051033497, ...","[9.456396e-05, 0.0030773198, -0.006812645, -0...."
2,5,2022-07-19 16:01:42.998,512.0,NT AUTHORITY\SYSTEM,Medium,0,55,1236,1,0,0,4,1,"[-0.00053622725, 0.00023643136, 0.0051033497, ...","[-0.0086196875, 0.003665738, 0.0051898835, 0.0..."
3,1,2022-07-19 16:01:42.937,512.0,NT AUTHORITY\SYSTEM,System,0,0,1237,2,1,1,4,1,"[-0.00053622725, 0.00023643136, 0.0051033497, ...","[9.456396e-05, 0.0030773198, -0.006812645, -0...."
4,5,2022-07-19 16:01:42.847,4920.0,NT AUTHORITY\SYSTEM,Medium,0,55,1236,1,0,0,4,1,"[-0.00053622725, 0.00023643136, 0.0051033497, ...","[-0.0086196875, 0.003665738, 0.0051898835, 0.0..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
329311,1,2022-07-19 16:01:46.219,1892.0,WIN-32-H1\aalsahee,Medium,1,0,1457,2,1,1,4,1,"[-0.007139015, 0.0012410306, -0.0071767163, -0...","[9.456396e-05, 0.0030773198, -0.006812645, -0...."
329312,1,2022-07-19 16:01:46.204,2872.0,WIN-32-H1\aalsahee,Medium,1,0,1191,4,1,1,4,1,"[0.00973555, -0.009780383, -0.00649949, 0.0027...","[9.456396e-05, 0.0030773198, -0.006812645, -0...."
329313,5,2022-07-19 16:01:46.017,1504.0,NT AUTHORITY\SYSTEM,Medium,1,55,1236,1,0,0,4,1,"[-0.0051562428, -0.0066683376, -0.007776835, 0...","[-0.0086196875, 0.003665738, 0.0051898835, 0.0..."
329314,5,2022-07-19 16:01:46.001,1404.0,NT AUTHORITY\SYSTEM,Medium,1,55,1236,1,0,0,4,1,"[0.007088797, -0.00156793, 0.007947499, -0.009...","[-0.0086196875, 0.003665738, 0.0051898835, 0.0..."


In [295]:
df.drop("utc_time", axis=1, inplace=True)

In [296]:
def parse_embedding(value):
    if isinstance(value, str):
        return np.array(eval(value))
    elif isinstance(value, list) or isinstance(value, np.ndarray):
        return np.array(value)
    else:
        return np.zeros(128)  
embedding_cols = ["image_embedding", "parent_image_embedding"]
for col in embedding_cols:
    df[col] = df[col].apply(parse_embedding) 
    df = df.join(pd.DataFrame(df[col].tolist(), index=df.index, 
                              columns=[f"{col}_{i}" for i in range(len(df[col][0]))]))
    df.drop(columns=[col], inplace=True) 
categorical_columns = ["user", "integrity_level", "command_line_encoded", "parent_image_encoded"]
label_encoders = {}
for col in categorical_columns:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col].astype(str))
    label_encoders[col] = le
X = df.drop(columns=["label"]) 
y = df["label"]  

In [297]:
df

,event_id,process_id,user,integrity_level,label,parent_image_encoded,command_line_encoded,command_length,hash_exists,has_parent_image,...,parent_image_embedding_90,parent_image_embedding_91,parent_image_embedding_92,parent_image_embedding_93,parent_image_embedding_94,parent_image_embedding_95,parent_image_embedding_96,parent_image_embedding_97,parent_image_embedding_98,parent_image_embedding_99
0,5,1688.0,2,2,0,51,265,1,0,0,...,0.001088,-0.001576,0.002197,-0.007882,-0.002717,0.002663,0.005347,-0.002392,-0.009510,0.004506
1,1,1688.0,2,3,0,0,266,2,1,1,...,-0.004509,0.005702,0.009180,-0.004100,0.007965,0.005375,0.005879,0.000513,0.008213,-0.007019
2,5,512.0,2,2,0,51,265,1,0,0,...,0.001088,-0.001576,0.002197,-0.007882,-0.002717,0.002663,0.005347,-0.002392,-0.009510,0.004506
3,1,512.0,2,3,0,0,266,2,1,1,...,-0.004509,0.005702,0.009180,-0.004100,0.007965,0.005375,0.005879,0.000513,0.008213,-0.007019
4,5,4920.0,2,2,0,51,265,1,0,0,...,0.001088,-0.001576,0.002197,-0.007882,-0.002717,0.002663,0.005347,-0.002392,-0.009510,0.004506
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
329311,1,1892.0,3,2,1,0,510,2,1,1,...,-0.004509,0.005702,0.009180,-0.004100,0.007965,0.005375,0.005879,0.000513,0.008213,-0.007019
329312,1,2872.0,3,2,1,0,215,4,1,1,...,-0.004509,0.005702,0.009180,-0.004100,0.007965,0.005375,0.005879,0.000513,0.008213,-0.007019
329313,5,1504.0,2,2,1,51,265,1,0,0,...,0.001088,-0.001576,0.002197,-0.007882,-0.002717,0.002663,0.005347,-0.002392,-0.009510,0.004506
329314,5,1404.0,2,2,1,51,265,1,0,0,...,0.001088,-0.001576,0.002197,-0.007882,-0.002717,0.002663,0.005347,-0.002392,-0.009510,0.004506


In [298]:
df

,event_id,process_id,user,integrity_level,label,parent_image_encoded,command_line_encoded,command_length,hash_exists,has_parent_image,...,parent_image_embedding_90,parent_image_embedding_91,parent_image_embedding_92,parent_image_embedding_93,parent_image_embedding_94,parent_image_embedding_95,parent_image_embedding_96,parent_image_embedding_97,parent_image_embedding_98,parent_image_embedding_99
0,5,1688.0,2,2,0,51,265,1,0,0,...,0.001088,-0.001576,0.002197,-0.007882,-0.002717,0.002663,0.005347,-0.002392,-0.009510,0.004506
1,1,1688.0,2,3,0,0,266,2,1,1,...,-0.004509,0.005702,0.009180,-0.004100,0.007965,0.005375,0.005879,0.000513,0.008213,-0.007019
2,5,512.0,2,2,0,51,265,1,0,0,...,0.001088,-0.001576,0.002197,-0.007882,-0.002717,0.002663,0.005347,-0.002392,-0.009510,0.004506
3,1,512.0,2,3,0,0,266,2,1,1,...,-0.004509,0.005702,0.009180,-0.004100,0.007965,0.005375,0.005879,0.000513,0.008213,-0.007019
4,5,4920.0,2,2,0,51,265,1,0,0,...,0.001088,-0.001576,0.002197,-0.007882,-0.002717,0.002663,0.005347,-0.002392,-0.009510,0.004506
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
329311,1,1892.0,3,2,1,0,510,2,1,1,...,-0.004509,0.005702,0.009180,-0.004100,0.007965,0.005375,0.005879,0.000513,0.008213,-0.007019
329312,1,2872.0,3,2,1,0,215,4,1,1,...,-0.004509,0.005702,0.009180,-0.004100,0.007965,0.005375,0.005879,0.000513,0.008213,-0.007019
329313,5,1504.0,2,2,1,51,265,1,0,0,...,0.001088,-0.001576,0.002197,-0.007882,-0.002717,0.002663,0.005347,-0.002392,-0.009510,0.004506
329314,5,1404.0,2,2,1,51,265,1,0,0,...,0.001088,-0.001576,0.002197,-0.007882,-0.002717,0.002663,0.005347,-0.002392,-0.009510,0.004506


In [318]:
print(df.isnull().sum())  # Check for missing values
print(np.isinf(df).sum())  # Check for infinite values
print(np.isnan(df).sum())  # Check for NaNs

event_id                     0
process_id                   0
user                         0
integrity_level              0
label                        0
                            ..
parent_image_embedding_95    0
parent_image_embedding_96    0
parent_image_embedding_97    0
parent_image_embedding_98    0
parent_image_embedding_99    0
Length: 212, dtype: int64
event_id                     0
process_id                   0
user                         0
integrity_level              0
label                        0
                            ..
parent_image_embedding_95    0
parent_image_embedding_96    0
parent_image_embedding_97    0
parent_image_embedding_98    0
parent_image_embedding_99    0
Length: 212, dtype: int64
event_id                     0
process_id                   0
user                         0
integrity_level              0
label                        0
                            ..
parent_image_embedding_95    0
parent_image_embedding_96    0
parent_image_embed

In [315]:
df["process_id"].fillna(df["process_id"].median(), inplace=True)  

<ipython-input-315-f1305109c6ff>:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["process_id"].fillna(df["process_id"].median(), inplace=True)  # Replace NaN with median


In [316]:
print(df["label"].unique()) 

[0 1]


In [317]:
df["label"] = df["label"].replace([-1, None], 0)

In [319]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train = np.nan_to_num(X_train, nan=0.0, posinf=1.0, neginf=-1.0)
X_test = np.nan_to_num(X_test, nan=0.0, posinf=1.0, neginf=-1.0)
y_train = np.nan_to_num(y_train, nan=0.0, posinf=1.0, neginf=-1.0)
y_test = np.nan_to_num(y_test, nan=0.0, posinf=1.0, neginf=-1.0)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [307]:
from tensorflow.keras.optimizers import Adam

In [321]:
model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    layers.Dense(32, activation='relu'),
    layers.Dense(1, activation='sigmoid')  
])

model.compile(optimizer=Adam(learning_rate=0.0001), 
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), 
              metrics=['accuracy'])

model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_test, y_test))


Epoch 1/5
8233/8233 ━━━━━━━━━━━━━━━━━━━━ 19s 2ms/step - accuracy: 0.9626 - loss: 0.1508 - val_accuracy: 0.9673 - val_loss: 0.1327
Epoch 2/5
8233/8233 ━━━━━━━━━━━━━━━━━━━━ 17s 2ms/step - accuracy: 0.9679 - loss: 0.1307 - val_accuracy: 0.9670 - val_loss: 0.1316
Epoch 3/5
8233/8233 ━━━━━━━━━━━━━━━━━━━━ 17s 2ms/step - accuracy: 0.9671 - loss: 0.1317 - val_accuracy: 0.9674 - val_loss: 0.1335
Epoch 4/5
8233/8233 ━━━━━━━━━━━━━━━━━━━━ 17s 2ms/step - accuracy: 0.9677 - loss: 0.1305 - val_accuracy: 0.9676 - val_loss: 0.1313
Epoch 5/5
8233/8233 ━━━━━━━━━━━━━━━━━━━━ 18s 2ms/step - accuracy: 0.9680 - loss: 0.1295 - val_accuracy: 0.9676 - val_loss: 0.1318


In [322]:
loss, accuracy = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {accuracy:.4f}")

2059/2059 ━━━━━━━━━━━━━━━━━━━━ 3s 1ms/step - accuracy: 0.9675 - loss: 0.1320
Test Accuracy: 0.9676


In [324]:
import numpy as np

sample_index = 2  
sample_data = X_test[sample_index].reshape(1, -1)  

prediction_prob = model.predict(sample_data)
prediction_label = (prediction_prob > 0.5).astype(int)  

print(f"Predicted Probability: {prediction_prob[0][0]:.4f}")
print(f"Predicted Label: {prediction_label[0][0]}")
print(f"Actual Label: {y_test[sample_index]}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Predicted Probability: 0.0251
Predicted Label: 0
Actual Label: 0
